<a href="https://colab.research.google.com/github/rponeconsultoria/portfoliorp/blob/main/Automa%C3%A7%C3%A3o_Documentos_GAppsScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/**
 * Script de automação para gerar documentos personalizados a partir de respostas de formulário
 * e dados de referência de clientes. Gera um documento com placeholders substituídos,
 * converte em PDF e envia por e-mail.
 *
 * Observação: IDs de planilhas, documentos e pastas foram mantidos genéricos.
 */

function gerarDocumento() {
  // Acesso às planilhas de respostas e referência
  const sheetRespostas = SpreadsheetApp.openById('ID_PLANILHA_RESPOSTAS').getSheetByName('Comodato Padrão Jales');
  const sheetRef = SpreadsheetApp.openById('ID_PLANILHA_REF').getSheetByName('Tabela Ref - Clientes');

  if (!sheetRespostas || !sheetRef) {
    Logger.log("Erro: Uma das planilhas não foi encontrada.");
    return;
  }

  // Verifica se há novas respostas
  const lastRow = sheetRespostas.getLastRow();
  if (lastRow < 2) {
    Logger.log("Nenhuma resposta encontrada no formulário!");
    return;
  }

  // Captura a última linha de respostas
  const respostas = sheetRespostas.getRange(lastRow, 1, 1, sheetRespostas.getLastColumn()).getValues()[0];
  const codigoCliente = respostas[0]; // Código do Cliente

  // Busca os dados do cliente na planilha de referência
  const dadosClientes = sheetRef.getDataRange().getValues();
  const clienteData = dadosClientes.find((row, index) => index > 0 && row[0] === codigoCliente);

  if (!clienteData) {
    Logger.log("Código do Cliente não encontrado na planilha de referência.");
    return;
  }

  // Mapeamento dos campos para preenchimento do template
  const placeholders = {
    'RAZAO_SOCIAL': clienteData[1],
    'CNPJ_VAR': clienteData[2],
    'NOME_FANTASIA': clienteData[3],
    'CIDADE_ESTADO_VAR': `${clienteData[8]} - ${clienteData[7]}`,
    'ENDERECO_NUMERO': respostas[2],
    'BAIRRO': respostas[3],
    'CEP': respostas[4],
    'TANQUE_VAR': respostas[5],
    'FILTRO_VAR': respostas[6],
    'BOMBA_VAR': respostas[7],
    'MEDIDOR_VAR': respostas[8],
    'OUTROS_VAR': respostas[9],
    'END_ENTREGA_VAR': respostas[10],
    'END_SEDE_VAR': respostas[11],
    'BAIRRO_SEDE': respostas[12],
    'CEP_SEDE': respostas[13],
    'EMAIL_PRINCIPAL': respostas[14],
    'TEL_PRINCIPAL': respostas[15],
    'DATA_VAR': respostas[16],
    'MES_VAR': respostas[17],
    'ANO_VAR': respostas[18],
    'REP_LEGAL': respostas[19],
    'TEST_UM': respostas[20],
    'DOCTEST_UM': respostas[21],
    'TEST_DOIS': respostas[22],
    'DOCTEST_DOIS': respostas[23]
  };

  // ID do documento modelo (com placeholders do tipo {{CAMPO}})
  const templateDocId = 'ID_TEMPLATE_DOCUMENTO';

  try {
    // Cria cópia do documento base e substitui os campos
    const copiaDoc = DriveApp.getFileById(templateDocId).makeCopy(`Cópia - ${new Date().toISOString()}`);
    const docFile = DocumentApp.openById(copiaDoc.getId());
    const body = docFile.getBody();

    // Substitui todos os placeholders no documento
    for (const key in placeholders) {
      body.replaceText(`{{${key}}}`, placeholders[key] || '');
    }

    docFile.saveAndClose();

    // Converte documento em PDF e salva em pasta
    const pdf = copiaDoc.getAs('application/pdf');
    const nomePdf = `Comodato ${placeholders['RAZAO_SOCIAL']}.pdf`;
    const pastaDestino = DriveApp.getFolderById('ID_PASTA_DESTINO');
    const arquivoPdf = pastaDestino.createFile(pdf);
    arquivoPdf.setName(nomePdf);

    // Envia por e-mail
    MailApp.sendEmail({
      to: 'email_destinatario@dominio.com',
      subject: `Comodato para Assinatura - ${placeholders['RAZAO_SOCIAL']}`,
      body: `Olá,\n\nSegue o comodato do cliente ${placeholders['RAZAO_SOCIAL']} para assinatura.\n\nAtenciosamente,\nEquipe`,
      attachments: [arquivoPdf]
    });

    Logger.log("Documento gerado, convertido e enviado com sucesso.");
  } catch (e) {
    Logger.log("Erro durante o processamento do documento: " + e.message);
  }
}
